In [3]:
# Load the auto reload extension to automatically reload modules when files on disk are updated
%load_ext autoreload
# it will automatically be reloaded without the need to restart the kernel.
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [4]:
import boto3
import geopandas as gpd
from io import BytesIO
import pandas as pd
from folium import GeoJson
import folium
import numpy as np
from tqdm import tqdm
from glob import glob
#import local modules
import sys
sys.path.append("/agrilearn_app/agrilearn/")
from agrilearn.utils import s3_utils, str_utils
from agrilearn.mvp import gpkg_utils

### Global Variables

In [5]:
DATASET_PATH = "/agrilearn_app/datasets/v1/geopackage/cana_fields_filtered.gpkg"
OUTPUT_DATASET_PATH = "/agrilearn_app/datasets/v1/geopackage/cana_fields_checked.gpkg"

# 1. Read Datasets

In [6]:
df = gpd.read_file(DATASET_PATH)

In [7]:
df.head()

,monitoring_class,period,fonte,cultura_2,state,area,meso,obs_extra,eopath_location,start_season,end_season,peak_start,peak_end,sentinel_eopatch_current,contour_score,contour_selected_timestamp,field_id,compac_index,dataset_part,geometry
0,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,2021-04-01,2020-04-01,2021-04-01,start_2020-04-01_end_2021-04-01_monitoring_cla...,1.0,2020-04-27,0_2,1.433508,train,"POLYGON ((-48.34483 -8.91348, -48.34412 -8.913..."
1,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,2021-04-01,2020-04-01,2021-04-01,start_2020-04-01_end_2021-04-01_monitoring_cla...,1.0,2020-04-27,0_3,1.705125,train,"POLYGON ((-48.35083 -8.91933, -48.35008 -8.917..."
2,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,2021-04-01,2020-04-01,2021-04-01,start_2020-04-01_end_2021-04-01_monitoring_cla...,1.0,2020-04-27,0_5,1.501758,train,"POLYGON ((-48.343 -8.91709, -48.34291 -8.91718..."
3,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,2021-04-01,2020-04-01,2021-04-01,start_2020-04-01_end_2021-04-01_monitoring_cla...,1.0,2020-04-27,0_6,1.493624,train,"POLYGON ((-48.34644 -8.9189, -48.34626 -8.9190..."
4,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,2021-04-01,2020-04-01,2021-04-01,start_2020-04-01_end_2021-04-01_monitoring_cla...,1.0,2020-04-27,0_7,1.746982,train,"POLYGON ((-48.34119 -8.91971, -48.34119 -8.919..."


In [8]:
df.shape

(35276, 20)

In [9]:
df['dataset_part'].value_counts(normalize=True)*100

dataset_part
train    91.362399
test      4.510149
val       4.127452
Name: proportion, dtype: float64

In [10]:
report_train_val_test = df.groupby(['monitoring_class', 'dataset_part'], as_index=False)\
                            .agg({'period':'count'})\
                            .sort_values(['monitoring_class', 'period'], ascending=[False, False])
report_train_val_test

,monitoring_class,dataset_part,period
1,SUGAR_CANE,train,32229
0,SUGAR_CANE,test,1591
2,SUGAR_CANE,val,1456


## 2. Check Eopatches from Local Path

In [11]:
EOPATCH_LOCAL_PATH="/agrilearn_app/datasets/eopatchs/processed/cana/**"
eopatches_path = [f for f in glob(EOPATCH_LOCAL_PATH, recursive=True) if f.endswith('/eopatch_0_col-0_row-0')]

In [12]:
len(eopatches_path)

35276

In [13]:
df['local_eopatch_path'] = eopatches_path

In [14]:
df['local_eopatch_path'].str.split('/start_').str[0].value_counts()

local_eopatch_path
/agrilearn_app/datasets/eopatchs/processed/cana/train    32229
/agrilearn_app/datasets/eopatchs/processed/cana/test      1591
/agrilearn_app/datasets/eopatchs/processed/cana/val       1456
Name: count, dtype: int64

## 2.1 Check total of missing images based on frequence time

In [15]:
dic_missing_images = gpkg_utils.get_number_of_missing_images(EOPATCH_IDS=df['local_eopatch_path'].values, freq='5D')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35276/35276 [09:14<00:00, 63.65it/s]


In [16]:
df_missing_images = pd.DataFrame(dic_missing_images).T.reset_index(names='path')

In [17]:
df_missing_images['status_missing_images'].value_counts()

status_missing_images
OK    35276
Name: count, dtype: int64

In [18]:
df_missing_images

,path,total_imagens_do_intervalo,total_de_imagem,status_missing_images
0,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
1,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
2,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
3,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
4,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
...,...,...,...,...
35271,/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK
35272,/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK
35273,/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK
35274,/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK


In [19]:
df_missing_images.to_csv('/agrilearn_app/datasets/v1/csvs/get_number_of_missing_images_based_freq5D.csv', 
                 sep=';', 
                 float_format='%.2f',
                 index=False)

In [20]:
df_missing_images.head()

,path,total_imagens_do_intervalo,total_de_imagem,status_missing_images
0,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
1,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
2,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
3,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK
4,/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK


In [21]:
df['total_imagens_do_intervalo'] = df_missing_images['total_imagens_do_intervalo']
df['total_de_imagem'] = df_missing_images['total_de_imagem']
df['status_missing_images'] = df_missing_images['status_missing_images']

## 2.2 Check time series missing

In [22]:
dic_get_time_series_missing = gpkg_utils.get_time_series_missing(EOPATCH_IDS=df['local_eopatch_path'].values)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35276/35276 [03:43<00:00, 158.02it/s]


In [23]:
df_time_series_missing = pd.DataFrame(dic_get_time_series_missing).T.reset_index(names='path')
df_time_series_missing['mean_dates_diff'] = df_time_series_missing['mean_dates_diff'].astype("float")

In [24]:
df_time_series_missing['status_series_missing'].value_counts()

status_series_missing
OK    35276
Name: count, dtype: int64

In [25]:
df_time_series_missing.to_csv('/agrilearn_app/datasets/v1/csvs/get_time_series_missing.csv', 
                      sep=';', 
                      index=False,
                      float_format='%.2f')

In [26]:
df['days_gap'] = df_time_series_missing['days_gap']
df['dates_diff'] = df_time_series_missing['dates_diff']
df['mean_dates_diff'] = df_time_series_missing['mean_dates_diff'] 
df['status_series_missing'] = df_time_series_missing['status_series_missing']

In [35]:
df.sort_values('total_de_imagem', ascending=True)

,monitoring_class,period,fonte,cultura_2,state,area,meso,obs_extra,eopath_location,start_season,...,dataset_part,geometry,local_eopatch_path,total_imagens_do_intervalo,total_de_imagem,status_missing_images,days_gap,dates_diff,mean_dates_diff,status_series_missing
29391,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,GO,1205051.0,552,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,train,"POLYGON ((-48.84518 -15.31868, -48.84499 -15.3...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29364,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.86641 -15.18714, -48.86674 -15.1...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29363,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.86921 -15.17908, -48.86886 -15.1...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29362,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.877 -15.18315, -48.87893 -15.180...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29361,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.88092 -15.17779, -48.8809 -15.17...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,TO,6773929.0,61,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.10724 -9.18087, -48.10684 -9.181...",/agrilearn_app/datasets/eopatchs/processed/can...,74,146,OK,"[2020-04-01 00:00:00, 2020-04-04 00:00:00, 202...","[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, ...",2.52,OK
11802,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,SP,800580.0,371,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-48.41335 -22.68265, -48.41335 -22.6...",/agrilearn_app/datasets/eopatchs/processed/can...,74,146,OK,"[2021-04-01 00:00:00, 2021-04-04 00:00:00, 202...","[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, ...",2.52,OK
21669,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_reformada,MS,2709113.0,512,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,train,"POLYGON ((-52.96281 -18.94254, -52.96272 -18.9...",/agrilearn_app/datasets/eopatchs/processed/can...,74,147,OK,"[2021-04-01 00:00:00, 2021-04-04 00:00:00, 202...","[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, ...",2.50,OK
21668,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_reformada,MS,2709113.0,512,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,train,"POLYGON ((-52.95775 -18.94035, -52.95765 -18.9...",/agrilearn_app/datasets/eopatchs/processed/can...,74,147,OK,"[2021-04-01 00:00:00, 2021-04-04 00:00:00, 202...","[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, ...",2.50,OK


In [36]:
df.sort_values('mean_dates_diff', ascending=True)

,monitoring_class,period,fonte,cultura_2,state,area,meso,obs_extra,eopath_location,start_season,...,dataset_part,geometry,local_eopatch_path,total_imagens_do_intervalo,total_de_imagem,status_missing_images,days_gap,dates_diff,mean_dates_diff,status_series_missing
19586,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,PR,414596.0,419,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-52.42101 -23.25342, -52.42046 -23.2...",/agrilearn_app/datasets/eopatchs/processed/can...,74,146,OK,"[2021-04-03 00:00:00, 2021-04-05 00:00:00, 202...","[2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, ...",2.50,OK
21669,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_reformada,MS,2709113.0,512,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,train,"POLYGON ((-52.96281 -18.94254, -52.96272 -18.9...",/agrilearn_app/datasets/eopatchs/processed/can...,74,147,OK,"[2021-04-01 00:00:00, 2021-04-04 00:00:00, 202...","[3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, ...",2.50,OK
19590,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_reformada,PR,363260.0,419,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-52.39634 -23.31467, -52.39646 -23.3...",/agrilearn_app/datasets/eopatchs/processed/can...,74,146,OK,"[2021-04-03 00:00:00, 2021-04-05 00:00:00, 202...","[2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, ...",2.50,OK
19589,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,PR,281035.0,419,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,train,"POLYGON ((-52.41703 -23.26802, -52.41719 -23.2...",/agrilearn_app/datasets/eopatchs/processed/can...,74,146,OK,"[2021-04-03 00:00:00, 2021-04-05 00:00:00, 202...","[2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, ...",2.50,OK
19588,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,PR,401225.0,419,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-52.41834 -23.26926, -52.41838 -23.2...",/agrilearn_app/datasets/eopatchs/processed/can...,74,146,OK,"[2021-04-03 00:00:00, 2021-04-05 00:00:00, 202...","[2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, ...",2.50,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29364,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.86641 -15.18714, -48.86674 -15.1...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29363,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.86921 -15.17908, -48.86886 -15.1...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29362,SUGAR_CANE,2022/2023,Agrosatélite - Canasat 2023/2024,cana_soca,GO,5909572.0,552,,start_2022-04-01_end_2023-04-01_monitoring_cla...,2022-04-01,...,train,"POLYGON ((-48.877 -15.18315, -48.87893 -15.180...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK
29372,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,GO,636018.0,552,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,train,"POLYGON ((-48.7815 -15.23141, -48.7815 -15.231...",/agrilearn_app/datasets/eopatchs/processed/can...,74,63,OK,"[2022-04-05 00:00:00, 2022-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 5, 5,...",5.81,OK


## 3. concat files and Save new GPKG

In [27]:
OUTPUT_DATASET_PATH

'/agrilearn_app/datasets/v1/geopackage/cana_fields_checked.gpkg'

In [28]:
df.to_file(OUTPUT_DATASET_PATH, driver='GPKG', engine='fiona')

In [29]:
df

,monitoring_class,period,fonte,cultura_2,state,area,meso,obs_extra,eopath_location,start_season,...,dataset_part,geometry,local_eopatch_path,total_imagens_do_intervalo,total_de_imagem,status_missing_images,days_gap,dates_diff,mean_dates_diff,status_series_missing
0,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-48.34483 -8.91348, -48.34412 -8.913...",/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK,"[2020-04-05 00:00:00, 2020-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.07,OK
1,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-48.35083 -8.91933, -48.35008 -8.917...",/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK,"[2020-04-05 00:00:00, 2020-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.07,OK
2,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-48.343 -8.91709, -48.34291 -8.91718...",/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK,"[2020-04-05 00:00:00, 2020-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.07,OK
3,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-48.34644 -8.9189, -48.34626 -8.9190...",/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK,"[2020-04-05 00:00:00, 2020-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.07,OK
4,SUGAR_CANE,2020/2021,Agrosatélite - Canasat 2021/2022,cana_soca,TO,3603383.0,58,,start_2020-04-01_end_2021-04-01_monitoring_cla...,2020-04-01,...,train,"POLYGON ((-48.34119 -8.91971, -48.34119 -8.919...",/agrilearn_app/datasets/eopatchs/processed/can...,74,72,OK,"[2020-04-05 00:00:00, 2020-04-10 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.07,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35271,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,GO,7609666.0,558,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,test,"POLYGON ((-50.96946 -19.31377, -50.97019 -19.3...",/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK,"[2022-04-04 00:00:00, 2022-04-09 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.37,OK
35272,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,GO,7609666.0,558,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,test,"POLYGON ((-50.96251 -19.31623, -50.96498 -19.3...",/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK,"[2022-04-04 00:00:00, 2022-04-09 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.37,OK
35273,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,GO,7609666.0,558,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,test,"POLYGON ((-50.9802 -19.31351, -50.9801 -19.313...",/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK,"[2022-04-04 00:00:00, 2022-04-09 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.37,OK
35274,SUGAR_CANE,2021/2022,Agrosatélite - Canasat 2022/23,cana_soca,GO,7609666.0,558,,start_2021-04-01_end_2022-04-01_monitoring_cla...,2021-04-01,...,test,"MULTIPOLYGON (((-50.95141 -19.31792, -50.95255...",/agrilearn_app/datasets/eopatchs/processed/can...,74,68,OK,"[2022-04-04 00:00:00, 2022-04-09 00:00:00, 202...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",5.37,OK
